<a href="https://colab.research.google.com/github/odarrieta/odarrieta/blob/main/Hackathon5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
!pip install xgboost


In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
import pandas as pd

# Cargar los datos de entrenamiento y prueba
survey_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Hachathon/Surveydata_train.csv')
travel_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Hachathon/Traveldata_train.csv')
survey_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Hachathon/Surveydata_test.csv')
travel_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Hachathon/Traveldata_test.csv')

# Combinar los datos de entrenamiento y prueba usando la columna ID como clave
train_data = pd.merge(survey_train, travel_train, on='ID')
test_data = pd.merge(survey_test, travel_test, on='ID')

# Mostrar las primeras filas de los datos combinados
print(train_data.head())
print(test_data.head())


         ID  Overall_Experience       Seat_Comfort Seat_Class  \
0  98800001                   0  Needs Improvement  Green Car   
1  98800002                   0               Poor   Ordinary   
2  98800003                   1  Needs Improvement  Green Car   
3  98800004                   0         Acceptable   Ordinary   
4  98800005                   1         Acceptable   Ordinary   

  Arrival_Time_Convenient           Catering  Platform_Location  \
0               Excellent          Excellent    Very Convenient   
1               Excellent               Poor  Needs Improvement   
2       Needs Improvement  Needs Improvement  Needs Improvement   
3       Needs Improvement                NaN  Needs Improvement   
4              Acceptable         Acceptable         Manageable   

  Onboard_Wifi_Service Onboard_Entertainment Online_Support  ...  \
0                 Good     Needs Improvement     Acceptable  ...   
1                 Good                  Poor           Good  ...   
2 

In [60]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer

# Imputar valores faltantes para variables numéricas y categóricas
num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')

# Listar columnas numéricas y categóricas
num_cols = train_data.select_dtypes(include=['float64', 'int64']).columns.drop('Overall_Experience')
cat_cols = train_data.select_dtypes(include=['object']).columns

# Ajustar imputadores solo a las columnas correctas
num_imputer.fit(train_data[num_cols])
cat_imputer.fit(train_data[cat_cols])

# Imputar valores faltantes en los datos de entrenamiento
train_data[num_cols] = num_imputer.transform(train_data[num_cols])
train_data[cat_cols] = cat_imputer.transform(train_data[cat_cols])

# Codificar variables categóricas con un codificador personalizado
class CustomLabelEncoder:
    def __init__(self):
        self.encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

    def fit(self, data):
        self.encoder.fit(data)

    def transform(self, data):
        return self.encoder.transform(data)

    def fit_transform(self, data):
        self.encoder.fit(data)
        return self.encoder.transform(data)

custom_encoders = {}
for col in cat_cols:
    custom_encoders[col] = CustomLabelEncoder()
    train_data[col] = custom_encoders[col].fit_transform(train_data[[col]])

# Escalar características numéricas
scaler = StandardScaler()
scaler.fit(train_data[num_cols])
train_data[num_cols] = scaler.transform(train_data[num_cols])

# Mostrar las primeras filas de los datos preprocesados de entrenamiento
print(train_data.head())


         ID  Overall_Experience  Seat_Comfort  Seat_Class  \
0 -1.732032                   0           4.0         0.0   
1 -1.731996                   0           5.0         1.0   
2 -1.731959                   1           4.0         0.0   
3 -1.731922                   0           0.0         1.0   
4 -1.731886                   1           0.0         1.0   

   Arrival_Time_Convenient  Catering  Platform_Location  Onboard_Wifi_Service  \
0                      1.0       1.0                4.0                   3.0   
1                      1.0       5.0                3.0                   3.0   
2                      4.0       4.0                3.0                   4.0   
3                      4.0       0.0                3.0                   0.0   
4                      0.0       0.0                2.0                   4.0   

   Onboard_Entertainment  Online_Support  ...  Cleanliness  Online_Boarding  \
0                    4.0             0.0  ...          4.0         

In [61]:
# Guardar los IDs originales
test_ids = test_data['ID']

# Imputar y codificar variables categóricas en los datos de prueba
test_data[num_cols] = num_imputer.transform(test_data[num_cols])
test_data[cat_cols] = cat_imputer.transform(test_data[cat_cols])
for col in cat_cols:
    test_data[col] = custom_encoders[col].transform(test_data[[col]])

# Reemplazar valores -1 en las columnas categóricas por la moda de cada columna
for col in cat_cols:
    mode_value = train_data[col].mode()[0]
    test_data[col] = test_data[col].replace(-1, mode_value)

# Escalar características numéricas en los datos de prueba
test_data[num_cols] = scaler.transform(test_data[num_cols])

# Mostrar las primeras filas de los datos preprocesados de prueba
print(test_data.head())


          ID  Seat_Comfort  Seat_Class  Arrival_Time_Convenient  Catering  \
0  38.642540           0.0         0.0                      0.0       0.0   
1  38.642577           2.0         1.0                      3.0       5.0   
2  38.642613           1.0         1.0                      1.0       1.0   
3  38.642650           0.0         0.0                      1.0       0.0   
4  38.642687           1.0         1.0                      2.0       1.0   

   Platform_Location  Onboard_Wifi_Service  Onboard_Entertainment  \
0                2.0                   4.0                    1.0   
1                2.0                   0.0                    5.0   
2                4.0                   1.0                    1.0   
3                4.0                   5.0                    0.0   
4                3.0                   1.0                    1.0   

   Online_Support  Ease_of_Online_Booking  ...  Cleanliness  Online_Boarding  \
0             3.0                     1.0 

In [62]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

# Definir el modelo de XGBoost
xgb_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

# Definir la rejilla de parámetros para el ajuste de hiperparámetros
param_dist = {
    'n_estimators': [312, 362, 413],
    'max_depth': [11, 14, 17],
    'learning_rate': [0.036, 0.035, 0.05],
    'subsample': [0.98, 0.11, 0.115, 1.3],
    'colsample_bytree': [0.3, 0.35, 0.4, 0.6],
    'gamma': [0.2, 0.3, 0.45],
    'reg_alpha': [0.9, 0.95, 1.4],
    'reg_lambda': [1.5, 2.2, 2.7]
}
X = train_data.drop(columns=['Overall_Experience', 'ID'])
y = train_data['Overall_Experience']

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, n_iter=50, cv=3, n_jobs=-1, verbose=2, random_state=42)

# Ajustar RandomizedSearchCV al conjunto de datos
random_search.fit(X, y)

# Mejor combinación de parámetros
best_params = random_search.best_params_
print("Mejores parámetros:", best_params)

# Entrenar el modelo optimizado
optimized_xgb_model = XGBClassifier(**best_params, random_state=42, use_label_encoder=False, eval_metric='logloss')
optimized_xgb_model.fit(X, y)

# Generar predicciones en los datos de prueba
X_test = test_data.drop(columns=['ID'])
test_data['Overall_Experience'] = optimized_xgb_model.predict(X_test)

# Crear el archivo de envío con los IDs originales
submission = pd.DataFrame({'ID': test_ids, 'Overall_Experience': test_data['Overall_Experience'].astype(int)})
submission.to_csv('submission_final_xgb.csv', index=False)

# Mostrar las primeras filas del archivo de envío
print(submission.head())


Fitting 3 folds for each of 50 candidates, totalling 150 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
39 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
39 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/xgboost/core.py", line 730, in inner_f
    return func(**kwargs)
  File "/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py", line 1519, in fit
    self._Booster = train(
  File "/usr/local/lib/python3.10/dist-packages/xgboost/core.py", line 730, in inner_f
 

Mejores parámetros: {'subsample': 0.98, 'reg_lambda': 2.2, 'reg_alpha': 0.95, 'n_estimators': 362, 'max_depth': 17, 'learning_rate': 0.035, 'gamma': 0.45, 'colsample_bytree': 0.6}
         ID  Overall_Experience
0  99900001                   1
1  99900002                   1
2  99900003                   1
3  99900004                   0
4  99900005                   1


In [56]:
from google.colab import files

# Asegúrate de que el nombre del archivo y la ruta sean correctos
files.download('submission_final_xgb.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>